In [1]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# DATA

In [2]:
app_train = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\app_train.csv')
app_test = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\app_test.csv')
prev_app = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\prev_app.csv')
pos_cash = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\pos_cash.csv')
bureau = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\bureau_balance_merge_bureau.csv')
ccb = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\credit_card_balance_final.csv')
inp = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\installments_payments_final.csv').drop(columns='Unnamed: 0')

In [3]:
inp.rename(columns={'SK_ID_CURR_': 'SK_ID_CURR'},inplace=True)
full_train = pd.merge(app_train, bureau, on= 'SK_ID_CURR', how = 'left')
full_train = pd.merge(full_train,prev_app, on = 'SK_ID_CURR', how = 'left')
full_train = pd.merge(full_train,inp, on = 'SK_ID_CURR', how = 'left')
full_train = pd.merge(full_train,ccb, on = 'SK_ID_CURR', how = 'left')
full_train = pd.merge(full_train,pos_cash, on = 'SK_ID_CURR', how = 'left')

# Preprocessed

In [4]:
def find_high_nulL_per(data, threshold = 70):
    '''Func to drop col with %null > threshold (default 70)
        Input : data_train
    '''
    null_check = data.isnull().sum() / data.shape[0] * 100
    null_check = null_check.drop(null_check[null_check == 0].index).sort_values(ascending= False).reset_index()
    drop_null_col =null_check[null_check[0] > threshold]['index'].tolist()
    return drop_null_col


In [5]:
def fill_nan(data):
    '''Input is X_ord_trained
        Output is all the things that train do and they is used for test
    '''

    # Fill nan
    ### Numeric data
    num_data = data._get_numeric_data().drop(columns = ['SK_ID_CURR','TARGET'])
    mean_num_data = num_data.mean()
    ### Categorical data
    cat_data = data[[col for col in data.columns if col not in num_data.columns and col not in ['SK_ID_CURR','TARGET']]]
    mode_cat_data = cat_data.mode().iloc[0]


    return mean_num_data,num_data.columns ,mode_cat_data, cat_data.columns
    

In [6]:
full_train.drop(columns = find_high_nulL_per(full_train),axis = 1,inplace=True)
app_test.drop(columns = find_high_nulL_per(full_train),axis = 1, inplace=True)
fill_num,num_col ,fill_cat,cat_col = fill_nan(full_train)

# Join both train and test

In [7]:
full_app = pd.concat([app_train, app_test], axis = 0)
# full_app[(full_app['TARGET'] == 0) | (full_app['TARGET'] == 1)]

In [8]:
full_data = pd.merge(full_app, bureau, on= 'SK_ID_CURR', how = 'left')
full_data = pd.merge(full_data,prev_app, on = 'SK_ID_CURR', how = 'left')
full_data = pd.merge(full_data,inp, on = 'SK_ID_CURR', how = 'left')
full_data = pd.merge(full_data,ccb, on = 'SK_ID_CURR', how = 'left')
full_data = pd.merge(full_data,pos_cash, on = 'SK_ID_CURR', how = 'left')

In [9]:
full_data[num_col] = full_data[num_col].fillna(fill_num)
full_data[cat_col] = full_data[cat_col].fillna(fill_cat)
full_train[num_col] = full_train[num_col].fillna(fill_num)
full_train[cat_col] = full_train[cat_col].fillna(fill_cat)

# Transformation

In [10]:
for i in cat_col:
    print(f'{i}  -  {full_train[i].unique()}')

NAME_CONTRACT_TYPE  -  ['Cash loans' 'Revolving loans']
CODE_GENDER  -  ['F' 'M' 'XNA']
FLAG_OWN_CAR  -  ['N' 'Y']
FLAG_OWN_REALTY  -  ['N' 'Y']
NAME_TYPE_SUITE  -  ['Family' 'Unaccompanied' 'Spouse, partner' 'Children' 'Other_A'
 'Group of people' 'Other_B']
NAME_INCOME_TYPE  -  ['State servant' 'Working' 'Pensioner' 'Commercial associate' 'Unemployed'
 'Student' 'Businessman' 'Maternity leave']
NAME_EDUCATION_TYPE  -  ['Higher education' 'Secondary / secondary special' 'Incomplete higher'
 'Lower secondary' 'Academic degree']
NAME_FAMILY_STATUS  -  ['Married' 'Civil marriage' 'Single / not married' 'Widow' 'Separated'
 'Unknown']
NAME_HOUSING_TYPE  -  ['House / apartment' 'Rented apartment' 'With parents'
 'Municipal apartment' 'Office apartment' 'Co-op apartment']
OCCUPATION_TYPE  -  ['Core staff' 'Laborers' 'Managers' 'Drivers' 'Sales staff'
 'Cleaning staff' 'Private service staff' 'Medicine staff'
 'Security staff' 'Accountants' 'Cooking staff' 'High skill tech staff'
 'Low-skill

In [11]:
# ordinal cols
ord_col = ['INS_ON_TIME_GRADE_','PAYMENT_GRADE_','FLAG_CONTACT']
X_ord = full_data[ord_col]

ins_on_time_grade_encode = {'Good': 0, 'Par' : 1, 'Sub_Par' : 2, 'Poor' : 3,'Very_Poor': 4}
X_ord['INS_ON_TIME_GRADE_'] = X_ord['INS_ON_TIME_GRADE_'].map(ins_on_time_grade_encode)

ins_on_time_grade_encode = {'Good': 0, 'Par' : 1, 'Sub_Par' : 2, 'Poor' : 3,'Very_Poor': 4}
X_ord['PAYMENT_GRADE_'] = X_ord['PAYMENT_GRADE_'].map(ins_on_time_grade_encode)

ins_on_time_grade_encode = {'Good': 0, 'Normal' : 1, 'Bad' : 2}
X_ord['FLAG_CONTACT'] = X_ord['FLAG_CONTACT'].map(ins_on_time_grade_encode)


C:\Users\Admin\AppData\Local\Temp\ipykernel_13400\2804226941.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_ord['INS_ON_TIME_GRADE_'] = X_ord['INS_ON_TIME_GRADE_'].map(ins_on_time_grade_encode)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13400\2804226941.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_ord['PAYMENT_GRADE_'] = X_ord['PAYMENT_GRADE_'].map(ins_on_time_grade_encode)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13400\2804226941.py:12: SettingWithCopyWarning: 
A value is trying to be s

In [12]:
# norminal cols
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()
X_one_hot = full_train[[col for col in cat_col if col not in ord_col]]
X_one_hot = one_hot_encoder.fit(X_one_hot)


# Fit and transform on full_train
X_one_hot_train = one_hot_encoder.fit_transform(full_train[[col for col in cat_col if col not in ord_col]])
X_onehot_data = one_hot_encoder.transform(full_data[[col for col in cat_col if col not in ord_col]])
# Convert to array and concatenate with other features

X_onehot_data = X_onehot_data.toarray()
X_cat_data = pd.DataFrame(X_onehot_data)
X_cat = pd.concat([X_ord, X_cat_data], axis=1)

In [13]:
def check_values(column):
    # Check for infinity
    has_infinity = np.any(np.isinf(column))
    
    # Check for values too large for float64
    too_large_values = np.any(np.abs(column) > np.finfo(np.float64).max)
    
    return has_infinity, too_large_values

# Iterate over columns and check for infinity and large values
def find_col(data):
    cols = []
    for col in data.columns:
        has_infinity, too_large_values = check_values(data[col])
        
        if has_infinity:
            print(f"Column '{col}' contains infinity.")
            cols.append(col)

        
        if too_large_values:
            print(f"Column '{col}' contains values too large for dtype('float64').")
            if col not in cols:
                cols.append(col)
    return cols

inf_col = find_col(full_train[num_col])

Column 'CAR_EMPLOYED_RATIO' contains infinity.
Column 'CAR_EMPLOYED_RATIO' contains values too large for dtype('float64').
Column 'PAYS_TO_INS_' contains infinity.
Column 'PAYS_TO_INS_' contains values too large for dtype('float64').


In [14]:
from sklearn.preprocessing import MinMaxScaler

X_num = full_train[num_col].drop(columns=inf_col, axis=1)
scaler = MinMaxScaler()
# fit with X_train
X_num = scaler.fit(X_num)


# Transform the full_data
X_num = pd.DataFrame(scaler.transform(full_data[[col for col in num_col if col not in inf_col]]))


In [15]:
full_set = pd.concat([X_num,X_cat],axis = 1)
full_set['SK_ID_CURR'] = full_data['SK_ID_CURR']
full_set['TARGET'] = full_data['TARGET']

# SPLIT

In [16]:
train = full_set[(full_set['TARGET'] == 0) | (full_set['TARGET'] == 1)]
test = full_set[~((full_set['TARGET'] == 0) | (full_set['TARGET'] == 1))]

In [17]:
train = train.set_index('SK_ID_CURR')
train.to_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\train.csv')

In [18]:
test = test.drop(columns='TARGET',axis = 1)
test = test.set_index('SK_ID_CURR')
test.to_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\test.csv')